# Basic Data Structure Observations
---
## Purpose:
In this notebook we're going to do our first exploration of our EPA, USDA, and Redfin data. Eventually, we will combine these three datasets to identify correlations between pollution and various indicators of wealth. To start though, it's important to evaluate each dataset and the potential issues that could arise from using them. These issues could include:
* **ethical concerns** <br> Such as considering how the data was collected, if/how consent was obtained, how the data will be used, and how it could cause harm or shift the balance of power.  <br><br>
* **technical concerns**  <br> such as considering the magnitude of the data, it's structure, data reliability, and how missing data should be treated.     

As we explore the data, a key ethical concern could involve how the findings would be use to enact legislative change, or adjust public opinion about particular groups. As such, <u>we strictly prohibit the aggregation of senstive population data</u> (race, religion, age, etc) with this study. Likewise this study should not be used to endorse proposed pollution legislation, without expansive supporting evidence.  A detailed *Terms of Use* clause is maintained [here](link) regarding how this work may be cited and/or used. Before using this research, you must agree to adhere to the Terms of Use and copyright limitations.  

### Package Installation and Versioning Requirments:
For questions regarding python version, package installations, and other functional requirements, see the *Read Me* file contained [here](link).

Now, let's review the data structure we have

In [25]:
#import necessary packages: 
import pandas as pd


#We'll create function to load the data
def load_base_data(): 
    """This function will load the raw EPA, USDA, and Redfin datasets and return them as a list. 
    """
    df_emissions = pd.read_json("data/nei.json",dtype={'COUNTY FIPS': str, "STATE FIPS": str})
    df_USDA = pd.read_excel("data/Unemployment.xlsx",header=4)
    df_Redfin = pd.read_csv("data/county_market_tracker.tsv000", sep = '\t')
    return(df_emissions, df_USDA, df_Redfin)


#and let's add a function to count missing data in the datasets (if any)
def check_nan(df): 
    """Checking for null values"""
    return(df.isnull().sum())

#Get the data
df_emissions, df_USDA, df_Redfin = load_base_data()

#highlight missing data if any
print(check_nan(df_emissions))

#view sample -- starting with emission data
df_emissions.info()

STATE              0
STATE FIPS         0
COUNTY             0
SECTOR             0
COUNTY FIPS        0
POLLUTANT          0
POLLUTANT TYPE     0
EMISSIONS          0
UNIT OF MEASURE    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101010 entries, 0 to 101009
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   STATE            101010 non-null  object
 1   STATE FIPS       101010 non-null  object
 2   COUNTY           101010 non-null  object
 3   SECTOR           101010 non-null  object
 4   COUNTY FIPS      101010 non-null  object
 5   POLLUTANT        101010 non-null  object
 6   POLLUTANT TYPE   101010 non-null  object
 7   EMISSIONS        101010 non-null  int64 
 8   UNIT OF MEASURE  101010 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.9+ MB


In [20]:
#Let's also take a sample to see the data contents
df_emissions.sample(4)


,STATE,STATE FIPS,COUNTY,SECTOR,COUNTY FIPS,POLLUTANT,POLLUTANT TYPE,EMISSIONS,UNIT OF MEASURE
46333,WA,53,Wahkiakum,Mobile - On-Road non-Diesel Heavy Duty Vehicles,069,Carbon Monoxide,CAP,18,TON
7854,TN,47,Henderson,Biogenics - Vegetation and Soil,077,Carbon Monoxide,CAP,1077,TON
8781,OH,39,Lawrence,Mobile - Non-Road Equipment - Gasoline,087,Carbon Monoxide,CAP,930,TON
36606,MA,25,Berkshire,Fuel Comb - Residential - Natural Gas,003,Carbon Monoxide,CAP,41,TON


In [33]:
# Okay, so we see 9 columns, including a unique ID (FIPS) for the state and counties in the emissions dataset. 
# Emissions are separated by sector, which could be interesting to probe differences in pollution contributors,
# and we have the county name and state abbreviation for each emission source. Now since we extracted the data
# from EPA's NEI, we know that the only pollutant considered is carbon monoxide, so we can ignore those columns
# in the future. 

# Now, the dataframe has no missing values, but that doesn't mean the dataframe contains all 
# counties in the US. We'll check on that for all datasets shortly. Also, while it's not explicitely stated, 
# this is 2020 emission data only. This will be a problem for our USDA data and you'll see why shortly.

# Let's explore that data now. 

In [34]:
#checking the USDA data: 
print(check_nan(df_USDA))

#view sample -- starting with emission data
df_USDA.info()

FIPS_Code                                     0
State                                         0
Area_Name                                     0
Rural_Urban_Continuum_Code_2013              58
Urban_Influence_Code_2013                    58
                                             ..
Employed_2022                                 4
Unemployed_2022                               4
Unemployment_rate_2022                        4
Median_Household_Income_2021                 83
Med_HH_Income_Percent_of_State_Total_2021    84
Length: 100, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3277 entries, 0 to 3276
Data columns (total 100 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   FIPS_Code                                  3277 non-null   int64  
 1   State                                      3277 non-null   object 
 2   Area_Name                                  32

In [27]:
df_USDA.sample(3)

,FIPS_Code,State,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
3043,53075,WA,"Whitman County, WA",4.0,5.0,0.0,19533.0,18750.0,783.0,4.0,...,23425.0,22400.0,1025.0,4.4,23236.0,22150.0,1086.0,4.7,53140.0,63.1
3084,54077,WV,"Preston County, WV",3.0,2.0,1.0,13228.0,12480.0,748.0,5.7,...,15400.0,14685.0,715.0,4.6,15600.0,14973.0,627.0,4.0,57174.0,111.8
2373,45023,SC,"Chester County, SC",1.0,1.0,1.0,16261.0,15418.0,843.0,5.2,...,13245.0,12490.0,755.0,5.7,13283.0,12717.0,566.0,4.3,46500.0,78.2


In [31]:
# So there's a lot to unpack here. First -- we have lots of columns, based on the variable of interest, and the 
# year of the data. Now there are some really awesome features in this dataset including income, 
# unemployment rates, labor force size, location, and rural/urban designation code (Continuum Code). 

# After a few minutes some issues should be apparent. First, we don't have income data for 2020, and the last rural
# continuum code record was taken in 2013. These variables could be really interesting to probe in our analysis. 

# Since we could not find readily available data to compensate here, we plan to use both of these variables, but
# there's some pretty big assumptions here: 
# 1. We assume income changes between 2020 and 2021 are neglgible down to the county level. For some counties this 
#    could be vastly different from reality given the COVID pandemic's impact on local economies. 
# 2. We assume the rural urban continuum is reasonably close to what it was in 2013. Again this could be an issue 
#    as some regions substantial growth during this period, like Austin TX while other areas had swaths of emigration
#    (like Edenville, MI where a dam break forces residents to leave)


In [32]:
#Before we dive further into this, let's review the Redfin dataset

#checking the USDA data: 
print(check_nan(df_Redfin))

#view sample -- starting with emission data
df_Redfin.info()

period_begin                           0
period_end                             0
period_duration                        0
region_type                            0
region_type_id                         0
table_id                               0
is_seasonally_adjusted                 0
region                                 0
city                              563122
state                                  0
state_code                             0
property_type                          0
property_type_id                       0
median_sale_price                    685
median_sale_price_mom              52383
median_sale_price_yoy              69332
median_list_price                  43920
median_list_price_mom              81940
median_list_price_yoy              98455
median_ppsf                         7395
median_ppsf_mom                    58352
median_ppsf_yoy                    75335
median_list_ppsf                   44644
median_list_ppsf_mom               82833
median_list_ppsf

In [29]:
df_Redfin.sample(3)

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
407272,2014-03-01,2014-03-31,30,county,5,3217,f,"Rusk County, WI",NaN,Wisconsin,...,0.166667,NaN,NaN,NaN,0.00000,0.000000,0.00000,Wisconsin nonmetropolitan area,NaN,2022-01-09 14:29:56
111322,2020-03-01,2020-03-31,30,county,5,224,f,"Pinal County, AZ",NaN,Arizona,...,0.000000,0.090909,-0.034091,-0.284091,1.00000,0.500000,1.00000,"Phoenix, AZ",38060.0,2022-01-09 14:29:56
344753,2017-03-01,2017-03-31,30,county,5,1865,f,"Clark County, NV",NaN,Nevada,...,0.032810,0.149445,0.010556,-0.028017,0.13576,-0.022877,-0.00242,"Las Vegas, NV",29820.0,2022-01-09 14:29:56


Okay, so at a high-level we see that the Redfin dataset provides information on sale price, property type, 
location, number of listings, and the period when the sale occured. Notably, the state name is not 
abbreviated and we aren't given any FIPs to connect to the other datasets, so that's going to be 
an issue down the road. We also only want 2021 data (since our income data is from the same time period, 
and we assume emissions are the same in 2020 and 2021). 

Now that we've scratched the surface, we'd like to see if we can make more sense of the data graphically, so we can understand how factors like income, pollution, unemployment, and sale prices are distributed. 

To do this graphically, we need to some light data cleaning and grouping, which is covered in the next notebook. 

### Next Notebook: Data Cleaning

*Note: to limit the number of functions duplicated, all codebook functions will be saved in py files that can be imported to execute.*